In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import digamma
from sklearn.neighbors import KDTree, BallTree
import warnings
import os
import seaborn as sns
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
import numpy.linalg as la
from numpy import log
from scipy.special import digamma
from sklearn.neighbors import BallTree, KDTree
import sklearn

In [ ]:
def reset():
    f =  ['AMG', 'CAR', 'CF1', 'CF3', 'CF4',
            'Others', 'GLI', 'LIN', 'LIP', 'MAC', 'NTI', 'OXA', 'PAP', 'PEN', 'POL',
            'QUI', 'SUL', 'MV hours', 'Acinet$_{pc}$', 'Enterobac$_{pc}$', 'Enteroc$_{pc}$',
            'Pseud$_{pc}$', 'Staph$_{pc}$', 'Others$_{pc}$', 'ICU$_{hours}$',
            '# pat$_{atb}$', '# pat$_{MDR}$',
            'CAR$_{n}$', 'PAP$_{n}$', 
            'Others$_{n}$', 'QUI$_{n}$',
            'OXA$_{n}$', 'PEN$_{n}$', 'CF3$_{n}$', 'GLI$_{n}$',
            'CF4$_{n}$', 'SUL$_{n}$', 'NTI$_{n}$', 'LIN$_{n}$',
            'AMG$_{n}$', 'MAC$_{n}$', 'CF1$_{n}$', 'POL$_{n}$',
            'LIP$_{n}$', '# pat$_{tot}$' ,'Post change',
            'Insulin', 'Art nutrition', 'Sedation', 'Relax', 'Hepatic$_{fail}$',
            'Renal$_{fail}$', 'Coagulation$_{fail}$', 'Hemodynamic$_{fail}$',
            'Respiratory$_{fail}$', 'Multiorganic$_{fail}$',  '# transfusions',
            'Vasoactive drug', 'Dosis nems', 'Tracheo$_{hours}$', 'Ulcer$_{hours}$',
            'Hemo$_{hours}$', 'C01 PIVC 1',
            'C01 PIVC 2', 'C02 CVC - RJ',
            'C02 CVC - RS', 'C02 CVC - LS', 'C02 CVC - RF',
            'C02 CVC - LJ', 'C02 CVC - LF', '# catheters']


    tf =  ['discreta', 'discreta', 'discreta', 'discreta', 'discreta', 
                        'discreta', 'discreta', 'discreta', 'discreta', 'discreta',
                        'discreta', 'discreta', 'discreta', 'discreta', 'discreta',
                        'discreta', 'discreta', 'continua', 'discreta',  
                        'discreta', 'discreta', 'discreta', 'discreta', 'discreta', 'continua',
                        'continua', 'continua', 'continua',
                        'continua', 'continua', 'continua',
                        'continua', 'continua', 'continua', 'continua',
                        'continua', 'continua', 'continua', 'continua',
                        'continua', 'continua', 'continua', 'continua',
                        'continua', 'continua', 'discreta', 'discreta',
                        'discreta', 'discreta', 'discreta', 'discreta', 'discreta',
                        'discreta', 'discreta', 'discreta',
                        'discreta', 'continua', 'discreta', 'continua',
                        'continua', 'continua', 'continua',
                        'continua', 'continua', 'continua', 'continua', 'continua',
                        'continua', 'continua', 'continua','continua']
    
    return f, tf

### Functions to separate MDR and Non MDR patients ###########################

def prepare_amr(split_num, features, norm):
    
    X_train = np.load(f"../../DATA/BBCET_cult_ok/s{split_num}/X_train_tensor_{norm}.npy")
    y_train = pd.read_csv(f"../../DATA/BBCET_cult_ok/s{split_num}/y_train_tensor_{norm}.csv")
    
    y_train_aux = y_train[y_train.individualMRGerm != 666].reset_index(drop=True)
    y_train_aux = y_train_aux.groupby(by="Admissiondboid").sum().reset_index()
    amr = y_train_aux[y_train_aux.individualMRGerm != 0].index
    
    X_train_amr = X_train[amr]
    P, T, F = X_train.shape
    y_train_values = y_train[['individualMRGerm']].values.flatten()
    y_train_amr = y_train_values.reshape((P, T))
    y_train_amr = y_train_amr[amr]
    
    dfs = []
    for t in range(T):
        temp_df = pd.DataFrame(X_train_amr[:, t, :], columns=[f'{feature}_{t}' for feature in features])
        dfs.append(temp_df)
    final_df = pd.concat(dfs, axis=1)
    
    dls = []
    for t in range(T):
        temp_df = pd.DataFrame(y_train_amr[:, t], columns=[t])
        dls.append(temp_df)
    final_dl = pd.concat(dls, axis=1)
    
    return final_df, final_dl, T, F

def prepare_noamr(split_num, features, norm):
    
    X_train = np.load(f"../../DATA/BBCET_cult_ok/s{split_num}/X_train_tensor_{norm}.npy")
    y_train = pd.read_csv(f"../../DATA/BBCET_cult_ok/s{split_num}/y_train_tensor_{norm}.csv")

    y_train_aux = y_train[y_train.individualMRGerm != 666].reset_index(drop=True)
    y_train_aux = y_train_aux.groupby(by="Admissiondboid").sum().reset_index()
    noamr = y_train_aux[y_train_aux.individualMRGerm == 0].index
    
    X_train_noamr = X_train[noamr]
    P, T, F = X_train.shape
    y_train_values = y_train[['individualMRGerm']].values.flatten()
    y_train_noamr = y_train_values.reshape((P, T))
    y_train_noamr = y_train_noamr[noamr]
    
    dfs = [] 
    for t in range(T):
        temp_df = pd.DataFrame(X_train_noamr[:, t, :], columns=[f'{feature}_{t}' for feature in features])
        dfs.append(temp_df)
    final_df = pd.concat(dfs, axis=1)
    
    dls = [] 
    for t in range(T):
        temp_df = pd.DataFrame(y_train_noamr[:, t], columns=[t])
        dls.append(temp_df)
    final_dl = pd.concat(dls, axis=1)
    
    return final_df, final_dl, T, F

def population(split_num, features, norm):
    
    X_train = np.load(f"../../DATA/BBCET_cult_ok/s{split_num}/X_train_tensor_{norm}.npy")
    y_train = pd.read_csv(f"../../DATA/BBCET_cult_ok/s{split_num}/y_train_tensor_{norm}.csv")
    
    y_train_aux = y_train[y_train.individualMRGerm != 666].reset_index(drop=True)
    y_train_aux = y_train_aux.groupby(by="Admissiondboid").sum().reset_index()
    pop = y_train_aux[y_train_aux.individualMRGerm >= 0].index
    
    X_train_pop = X_train[pop]
    P, T, F = X_train.shape
    y_train_values = y_train[['individualMRGerm']].values.flatten()
    y_train_pop = y_train_values.reshape((P, T))
    y_train_pop = y_train_pop[pop]
    
    dfs = [] 
    for t in range(T):
        temp_df = pd.DataFrame(X_train_pop[:, t, :], columns=[f'{feature}_{t}' for feature in features])
        dfs.append(temp_df)
    final_df = pd.concat(dfs, axis=1)
    
    dls = [] 
    for t in range(T):
        temp_df = pd.DataFrame(y_train_pop[:, t], columns=[t])
        dls.append(temp_df)
    final_dl = pd.concat(dls, axis=1)
    
    return final_df, final_dl, T, F

############################################################

# UTILITY FUNCTIONS
def count_neighbors(tree, x, r):
    return tree.query_radius(x, r, count_only=True)

def add_noise(x, intens):#-10!!
    # small noise to break degeneracy, see doc.
    return x + intens * np.random.random_sample(x.shape)

def build_tree(points, val):
    if points.shape[1] >= val:
        return BallTree(points, metric='chebyshev')
    return KDTree(points, metric='chebyshev')

def query_neighbors(tree, x, k):
    return tree.query(x, k=k + 1)[0][:, k]

def avgdigamma(points, dvec):
    # This part finds number of neighbors in some radius in the marginal space
    # returns expectation value of <psi(nx)>
    tree = build_tree(points, val)
    dvec = dvec - 1e-15
    num_points = count_neighbors(tree, points, dvec)
    return np.mean(digamma(num_points))
## END UTILITY FUNCTIONS ENTROPY


###########################################################

### FUNCT D-D (MI y CMI) ###
def entropyd(sx, base=2):
    """ Discrete entropy estimator
        sx is a list of samples
    """
    unique, count = np.unique(sx, return_counts=True, axis=0)
    # Convert to float as otherwise integer division results in all 0 for proba.
    proba = count.astype(float) / len(sx)
    # Avoid 0 division; remove probabilities == 0.0 (removing them does not change the entropy estimate as 0 * log(1/0) = 0.
    proba = proba[proba > 0.0]
    return np.sum(proba * np.log(1. / proba)) / log(base)

#### MI. D-D

def centropyd(x, y, base=2):
    """ The classic K-L k-nearest neighbor continuous entropy estimator for the
        entropy of X conditioned on Y.
    """
    xy = np.c_[x, y]
    return entropyd(xy, base) - entropyd(y, base)


def midd(x, y, base=2):
    """ Discrete mutual information estimator
        Given a list of samples which can be any hashable object
    """
    assert len(x) == len(y), "Arrays should have same length"
    return entropyd(x, base) - centropyd(x, y, base)


#### Cond. D-D
def cmidd(x, y, z, base=2):
    """ Discrete mutual information estimator
        Given a list of samples which can be any hashable object
    """
    assert len(x) == len(y) == len(z), "Arrays should have same length"
    xz = np.c_[x, z]
    yz = np.c_[y, z]
    xyz = np.c_[x, y, z]
    return entropyd(xz, base) + entropyd(yz, base) - entropyd(xyz, base) - entropyd(z, base)

#############################################################################

## MI - C-D
def entropy(x, k, base=2):
    """ The classic K-L k-nearest neighbor continuous entropy estimator
        x should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
        if x is a one-dimensional scalar and we have four samples
    """

    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    x = np.asarray(x)
    n_elements, n_features = x.shape
    x = add_noise(x, intens)
    tree = build_tree(x, val)
    nn = query_neighbors(tree, x, k)
    const = digamma(n_elements) - digamma(k) + n_features * log(2)
    return (const + n_features * np.log(nn).mean()) / log(base)

def micd(x, y, k, base=2, warning=True):
    """ If x is continuous and y is discrete, compute mutual information
    """
    assert len(x) == len(y), "Arrays should have same length"
    entropy_x = entropy(x, k, base)
    y_unique, y_count = np.unique(y, return_counts=True, axis=0)
    y_proba = y_count / len(y)

    entropy_x_given_y = 0.
    for yval, py in zip(y_unique, y_proba):
        x_given_y = x[(y == yval).all(axis=1)]
        
        if k <= len(x_given_y) - 1:
            entropy_x_given_y += py * entropy(x_given_y, k, base)
        else:
            if warning:
                warnings.warn("Warning, after conditioning, on y={yval} insufficient data. "
                              "Assuming maximal entropy in this case.".format(yval=yval))
            entropy_x_given_y += py * entropy_x
    return abs(entropy_x - entropy_x_given_y)  # units already applied


def midc(x, y, k, base=2, warning=True):
    return micd(y, x, k, base, warning)

## END MI - C-D

## CMI - C-D
def mi(x, y, z, k, base=2, alpha=0):
    """ 
    Mutual information of x and y (conditioned on z if z is not None)
    x, y should be a list of vectors, e.g. x = [[1.3], [3.7], [5.1], [2.4]]
    if x is a one-dimensional scalar and we have four samples
    """
    assert len(x) == len(y), "Arrays should have same length"
    assert k <= len(x) - 1, "Set k smaller than num. samples - 1"
    x, y = np.asarray(x), np.asarray(y)
    x, y = x.reshape(x.shape[0], -1), y.reshape(y.shape[0], -1)
    x = add_noise(x, intens)
    y = add_noise(y, intens)
    points = [x, y]
    if z is not None:
        z = np.asarray(z)
        z = z.reshape(z.shape[0], -1)
        points.append(z)
    points = np.hstack(points)
    # Find nearest neighbors in joint space, p=inf means max-norm
    tree = build_tree(points, val)
    dvec = query_neighbors(tree, points, k)
    if z is None:
        a, b, c, d = avgdigamma(x, dvec), avgdigamma(
            y, dvec), digamma(k), digamma(len(x))
        if alpha > 0:
            d += lnc_correction(tree, points, k, alpha)
    else:
        xz = np.c_[x, z]
        yz = np.c_[y, z]
        a, b, c, d = avgdigamma(xz, dvec), avgdigamma(
            yz, dvec), avgdigamma(z, dvec), digamma(k)
    return (-a - b + c + d) / log(base)


def cmi(x, y, z, k, base=2):
    """ Mutual information of x and y, conditioned on z
        Legacy function. Use mi(x, y, z) directly.
    """
    return mi(x, y, z, k, base=base)
## END CMI - C-D

######################################## STEP 0 ########################################

def firstMI(X, y, k, variable_types, base=2):
    maxMI = 0
    indexMIMax = 0
    
    claves = X.columns
    for k in range(X.shape[1]):
        # Select column in y corresponding to the day of the feature in X (f.e: feature 'AMG_0' with label of day 0)
        y_col = y.iloc[:, k // len(variable_types)].values

        # Filter the values in X and y to avoid the 666
        mask = (X.iloc[:, k].values != 666) & (y_col != 666)
        X_filtered = X.iloc[:, k].values[mask].reshape(-1, 1)
        y_filtered = y_col[mask].reshape(-1, 1)
        
        if variable_types[k] == 'discreta':
            miValue = np.abs(midd(X_filtered, y_filtered, base=base))
        else:
            miValue = np.abs(micd(X_filtered, y_filtered, k, base=base))
        
        if miValue > maxMI:
            maxMI = miValue
            indexMIMax = k
            
    # Eliminate the first variable and add it to z
    key = X.columns[indexMIMax]
    z = X[key].values.reshape(-1, 1)
    X = X.drop(columns=[key])
    
    return X, z, key, maxMI

def myCondMI(X, y, z, k, variable_types, base=2):
    maxMI = 0
    indexMIMax = 0
    claves = X.columns
    for f in range(X.shape[1]):
        # Select column in y corresponding to the day of the feature in X (f.e: feature 'AMG_0' with label of day 0)
        y_col = y.iloc[:, f // len(variable_types)].values

        # Filter the values in X and y to avoid the 666
        mask = (X.iloc[:, f].values != 666) & (y_col != 666)
        X_filtered = X.iloc[:, f].values[mask].reshape(-1, 1)
        y_filtered = y_col[mask].reshape(-1, 1)
        z_filtered = z[mask]
                
        if variable_types[f] == 'discreta':
            miValue = np.abs(cmidd(X_filtered, y_filtered, z_filtered, base=base))
        else:
            miValue = np.abs(cmi(X_filtered, y_filtered, z_filtered, k, base=base))
        
        if miValue > maxMI:
            maxMI = miValue
            indexMIMax = f
            
    # Eliminate the first variable and add it to z
    key = X.columns[indexMIMax]
    z = np.append(z, X[key].values.reshape(-1, 1), axis=1)
    X = X.drop(columns=[key])
    
    return X, z, key, maxMI

In [ ]:
split_directory = "./Results_CMI"

# MDR patients

In [ ]:
results_amr = {}

k_n = 1
intens = 1e-15
val = 35

norm = '0robustNorm'

for split_num in range(1,4):
    features, tipos_variables = reset()
    final_df, final_dl, T, F = prepare_amr(split_num, features, norm)
    indexesSelected = []
    MIvalues = []
    
    for t in range(T):
        features, tipos_variables = reset()
 
        X_day = final_df.iloc[:, t*F:(t+1)*F]
        y_day = final_dl.iloc[:, [t]]

        for j in range(F):
            if j == 0:
                X_day, z, featureSelected, maxMI = firstMI(X_day, y_day, k_n, tipos_variables)
                maxMI = maxMI/(maxMI*10)
            else:
                X_day, z, featureSelected, maxMI = myCondMI(X_day, y_day, z, k_n, tipos_variables)
            
            feat = featureSelected.split("_"+str(t))[0]
            idx = features.index(feat)
            del features[idx]
            del tipos_variables[idx]
            
            featureSelected = featureSelected.split("_"+str(t))[0]
            indexesSelected.append(featureSelected)
            MIvalues.append(maxMI)
            
    results_df = pd.DataFrame({
        'Feature': indexesSelected,
        'MI Value': MIvalues
    })
    
    results_amr[f'results_df_{split_num}'] = results_df
    

In [ ]:
with open(os.path.join(split_directory, f"CMI_results_amr.pkl"), 'wb') as f:
    pickle.dump(results_amr, f)

# Non MDR patients

In [ ]:
results_noamr = {}
intens = 1e-13
val = 32

for split_num in range(1,4):
    features, tipos_variables = reset()
    final_df, final_dl, T, F = prepare_noamr(split_num, features, norm)
    indexesSelected = []
    MIvalues = []
    
    for t in range(T):
        features, tipos_variables = reset()
       
        X_day = final_df.iloc[:, t*F:(t+1)*F]
        y_day = final_dl.iloc[:, [t]]

        for j in range(F):
            if j == 0:
                X_day, z, featureSelected, maxMI = firstMI(X_day, y_day, k_n, tipos_variables)
                maxMI = maxMI/(maxMI*10)
            else:
                X_day, z, featureSelected, maxMI = myCondMI(X_day, y_day, z, k_n, tipos_variables)
            
            feat = featureSelected.split("_"+str(t))[0]
            idx = features.index(feat)
            del features[idx]
            del tipos_variables[idx]
            
            featureSelected = featureSelected.split("_"+str(t))[0]
            indexesSelected.append(featureSelected)
            MIvalues.append(maxMI)
            
    results_df = pd.DataFrame({
        'Feature': indexesSelected,
        'MI Value': MIvalues
    })
    
    results_noamr[f'results_df_{split_num}'] = results_df
    
    

In [ ]:
with open(os.path.join(split_directory, f"CMI_results_noamr.pkl"), 'wb') as f:
    pickle.dump(results_noamr, f)

## Population analysis

In [ ]:
results_pop = {}
intens = 1e-13
val = 32

for split_num in range(1,4):
    features, tipos_variables = reset()
    final_df, final_dl, T, F = population(split_num, features, norm)
    indexesSelected = []
    MIvalues = []
    
    for t in range(T):
        features, tipos_variables = reset()
        
        X_day = final_df.iloc[:, t*F:(t+1)*F]
        y_day = final_dl.iloc[:, [t]]

        for j in range(F):
            if j == 0:
                X_day, z, featureSelected, maxMI = firstMI(X_day, y_day, k_n, tipos_variables)
                maxMI = maxMI/(maxMI*10)
            else:
                X_day, z, featureSelected, maxMI = myCondMI(X_day, y_day, z, k_n, tipos_variables)
            
            feat = featureSelected.split("_"+str(t))[0]
            idx = features.index(feat)
            del features[idx]
            del tipos_variables[idx]
            
            featureSelected = featureSelected.split("_"+str(t))[0]
            indexesSelected.append(featureSelected)
            MIvalues.append(maxMI)
            
    results_df = pd.DataFrame({
        'Feature': indexesSelected,
        'MI Value': MIvalues
    })
    
    results_pop[f'results_df_{split_num}'] = results_df

In [ ]:
with open(os.path.join(split_directory, f"CMI_results_population.pkl"), 'wb') as f:
    pickle.dump(results_pop, f)